In [77]:
# Inport definitions
import pwiki
import mwparserfromhell
import re

from rdflib import Graph, URIRef, Literal, Namespace


In [78]:
# piwiki configuration
bulbabot = pwiki.wiki.Wiki(api_endpoint="https://bulbapedia.bulbagarden.net/w/api.php")

In [ ]:
# piwiki basic tests
print(bulbabot.exists("Bulbasaur"))

bulbasaur_test = bulbabot.parse(title="Bulbasaur")
print( bulbasaur_test)

In [ ]:
#Bulbasaur page content test
bulbasaur_test2 = bulbabot.page_text(title="Bulbasaur (Pokémon)")
bulbasaur_test2

In [136]:
# FUNCTION DEFINITION : pokémon infobox text extraction
def extract_infobox(content):
    match = re.search(r"{{Pokémon Infobox.*?}}", content, re.DOTALL)
    if match:
        return match.group(0)
    return None


def extract_region_infobox(content):
    match = re.search(r"{{RegionInfobox.*?}}", content, re.DOTALL)
    if match:
        return match.group(0)
    return None

def extract_character_infobox(content):
    match = re.search(r"{{Character Infobox.*?}}", content, re.DOTALL)
    if match:
        return match.group(0)
    return None

In [81]:
# infobox extration test
infobox = extract_infobox(bulbasaur_test2)
infobox

'{{Pokémon Infobox\n|name=Bulbasaur\n|jname=フシギダネ\n|tmname=Fushigidane\n|ndex=0001\n|type1=Grass\n|type2=Poison\n|category=Seed\n|height-ftin=2\'04"\n|height-m=0.7\n|weight-lbs=15.2\n|weight-kg=6.9\n|abilityn=d\n|ability1=Overgrow\n|abilityd=Chlorophyll\n|egggroupn=2\n|egggroup1=Monster\n|egggroup2=Grass\n|eggcycles=20\n|evtotal=1\n|evsa=1\n|expyield=64\n|oldexp=64\n|lv100exp=1,059,860\n|gendercode=31\n|color=Green\n|catchrate=45\n|body=08\n|generation=1\n|pokefordex=bulbasaur\n|friendship=70\n}}'

In [80]:
# FUNCTION DEFINITION : parsing of the infobox text 
def parse_infobox(infobox):
    properties = {}
    lines = infobox.split("\n")
    for line in lines:
        if line.startswith("|"):
            key_value = line[1:].split("=", 1)  # Remove leading `|` and split at `=`
            if len(key_value) == 2:
                key, value = key_value
                properties[key.strip()] = value.strip()
    return properties

In [82]:
# Infobox text parsing test 
properties = parse_infobox(infobox)
properties

{'name': 'Bulbasaur',
 'jname': 'フシギダネ',
 'tmname': 'Fushigidane',
 'ndex': '0001',
 'type1': 'Grass',
 'type2': 'Poison',
 'category': 'Seed',
 'height-ftin': '2\'04"',
 'height-m': '0.7',
 'weight-lbs': '15.2',
 'weight-kg': '6.9',
 'abilityn': 'd',
 'ability1': 'Overgrow',
 'abilityd': 'Chlorophyll',
 'egggroupn': '2',
 'egggroup1': 'Monster',
 'egggroup2': 'Grass',
 'eggcycles': '20',
 'evtotal': '1',
 'evsa': '1',
 'expyield': '64',
 'oldexp': '64',
 'lv100exp': '1,059,860',
 'gendercode': '31',
 'color': 'Green',
 'catchrate': '45',
 'body': '08',
 'generation': '1',
 'pokefordex': 'bulbasaur',
 'friendship': '70'}

In [83]:
pokemon_infobox_graph = Graph()

In [84]:
# FUNCTION DEFINITION : mapping of pokemon infobox properties to existing graph
def map_infobox_to_rdf(graph, properties):
    
    POKEMON = Namespace("http://example.org/pokemon/")
    RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

    pokemon_uri = URIRef(POKEMON[properties.get("name", "Unknown").replace(" ", "_")])
    graph.add((pokemon_uri, RDF.type, POKEMON.Pokémon))

    mappings = {
        "name": POKEMON.name,
        "jname": POKEMON.japaneseName,
        "tmname": POKEMON.trademarkedJapaneseName,
        "jtranslit": POKEMON.japaneseTransliteration,
        "category": POKEMON.category,
        "ndex": POKEMON.nationalDexNumber,
        "forme": POKEMON.numberOfForms,
        "type1": POKEMON.primaryType,
        "type2": POKEMON.secondaryType,
        "ability1": POKEMON.ability1,
        "ability2": POKEMON.ability2,
        "abilityd": POKEMON.hiddenAbility,
        "height-m": POKEMON.heightInMeters,
        "height-ftin": POKEMON.heightInFeetInches,
        "weight-kg": POKEMON.weightInKilograms,
        "weight-lbs": POKEMON.weightInPounds,
        "catchrate": POKEMON.catchRate,
        "gendercode": POKEMON.genderRatioCode,
        "egggroup1": POKEMON.eggGroup1,
        "egggroup2": POKEMON.eggGroup2,
        "color": POKEMON.color,
        "friendship": POKEMON.baseFriendship,
        "generation": POKEMON.generationIntroduced,
        "expyield": POKEMON.baseExperienceYield,
        "evtotal": POKEMON.evTotal,
        "evhp": POKEMON.evYieldHP,
        "evat": POKEMON.evYieldAttack,
        "evde": POKEMON.evYieldDefense,
        "evsa": POKEMON.evYieldSpecialAttack,
        "evsd": POKEMON.evYieldSpecialDefense,
        "evsp": POKEMON.evYieldSpeed
    }

    for i in range(2, 7):  # Forms 2 through 6
        form_suffix = f"form{i}"
        mappings.update({
            f"{form_suffix}": POKEMON[f"form{i}Name"],
            f"{form_suffix}type1": POKEMON[f"form{i}PrimaryType"],
            f"{form_suffix}type2": POKEMON[f"form{i}SecondaryType"],
            f"height-m{i}": POKEMON[f"form{i}HeightInMeters"],
            f"height-ftin{i}": POKEMON[f"form{i}HeightInFeetInches"],
            f"weight-kg{i}": POKEMON[f"form{i}WeightInKilograms"],
            f"weight-lbs{i}": POKEMON[f"form{i}WeightInPounds"],
        })

    for key, predicate in mappings.items():
        if key in properties and properties[key]:
            graph.add((pokemon_uri, predicate, Literal(properties[key])))

    return graph


In [85]:
pokemon_infobox_graph = map_infobox_to_rdf(pokemon_infobox_graph, properties=properties)
print(pokemon_infobox_graph.serialize(format="turtle"))

@prefix ns1: <http://example.org/pokemon/> .

ns1:Bulbasaur a ns1:Pokémon ;
    ns1:ability1 "Overgrow" ;
    ns1:baseExperienceYield "64" ;
    ns1:baseFriendship "70" ;
    ns1:catchRate "45" ;
    ns1:category "Seed" ;
    ns1:color "Green" ;
    ns1:eggGroup1 "Monster" ;
    ns1:eggGroup2 "Grass" ;
    ns1:evTotal "1" ;
    ns1:evYieldSpecialAttack "1" ;
    ns1:genderRatioCode "31" ;
    ns1:generationIntroduced "1" ;
    ns1:heightInFeetInches "2'04\"" ;
    ns1:heightInMeters "0.7" ;
    ns1:hiddenAbility "Chlorophyll" ;
    ns1:japaneseName "フシギダネ" ;
    ns1:name "Bulbasaur" ;
    ns1:nationalDexNumber "0001" ;
    ns1:primaryType "Grass" ;
    ns1:secondaryType "Poison" ;
    ns1:trademarkedJapaneseName "Fushigidane" ;
    ns1:weightInKilograms "6.9" ;
    ns1:weightInPounds "15.2" .




In [132]:
# FUNCTION DEFINITION : mapping of region infobox properties to existing graph
def map_region_infobox_to_rdf(graph, properties):

    REGION = Namespace("http://example.org/region/")
    RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

    region_uri = URIRef(REGION[properties.get("name", "Unknown").replace(" ", "_")])
    graph.add((region_uri, RDF.type, REGION.Region))

    mappings = {
        "regioncolor": REGION.colorTemplate,
        "name": REGION.name,
        "jname": REGION.japaneseName,
        "tmname": REGION.romanizedJapaneseName,
        "region": REGION.isRegion,
        "image": REGION.image,
        "size": REGION.imageSize,
        "caption": REGION.imageCaption,
        "introduction": REGION.introductionMedia,
        "professor": REGION.professor,
        "firstpartner": REGION.firstPartnerPokemon,
        "villain": REGION.villainousOrganization,
        "league": REGION.league,
        "location": REGION.leagueLocation,
        "pokedext": REGION.pokedex,
        "series": REGION.animationSeries,
        "season": REGION.animationSeason,
        "generation": REGION.generation,
        "games": REGION.games,
        "manga": REGION.mangaChapter,
    }

    for key, predicate in mappings.items():
        if key in properties and properties[key]:
            graph.add((region_uri, predicate, Literal(properties[key])))

    return graph

In [138]:
# FUNCTION DEFINITION : mapping of character infobox properties to existing graph
def map_character_infobox_to_rdf(graph, properties):
    CHARACTER = Namespace("http://example.org/character/")
    RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

    character_uri = URIRef(CHARACTER[properties.get("name", "Unknown").replace(" ", "_")])
    graph.add((character_uri, RDF.type, CHARACTER.Character))

    mappings = {
        "color": CHARACTER.color,
        "bordercolor": CHARACTER.borderColor,
        "corecolor": CHARACTER.coreColor,
        "name": CHARACTER.name,
        "jname": CHARACTER.japaneseName,
        "tmname": CHARACTER.romanizedJapaneseName,
        "jtranslit": CHARACTER.japaneseTransliteration,
        "text": CHARACTER.textColor,
        "sloganline": CHARACTER.sloganLine,
        "image": CHARACTER.image,
        "size": CHARACTER.imageSize,
        "caption": CHARACTER.imageCaption,
        "age": CHARACTER.age,
        "birthday": CHARACTER.birthday,
        "gender": CHARACTER.gender,
        "height": CHARACTER.height,
        "eyes": CHARACTER.eyeColor,
        "hair": CHARACTER.hairColor,
        "hometown": CHARACTER.hometown,
        "region": CHARACTER.region,
        "relatives": CHARACTER.relatives,
    }

    for key, predicate in mappings.items():
        if key in properties and properties[key]:
            graph.add((character_uri, predicate, Literal(properties[key])))

    return graph

In [111]:
# Get the list of pokemons
poke_list = bulbabot.page_text("List_of_Pokémon_by_National_Pokédex_number")
poke_list


'{{shortcut|3|Ndex|Olddex|Natdex}}\nThis is a list of Pokémon in the order dictated by the [[National Pokédex]], meaning that Pokémon from the [[Kanto]] region will appear first, followed by those from [[Johto]], [[Hoenn]], [[Sinnoh]], [[Unova]], [[Kalos]], [[Alola]], [[Galar]], [[Hisui]], and [[Paldea]]. As of the release of the [[Mochi Mayhem]] epilogue of [[The Indigo Disk]] expansion for {{g|Scarlet and Violet}}, there are 1025 Pokémon in total. Each region\'s set of Pokémon starts with its own set of [[first partner Pokémon]] and their [[evolution]]s, going in order of {{t|Grass}}, {{t|Fire}}, {{t|Water}}; the only exception is Unova, which begins with {{p|Victini}}, who is then followed by the first partner Pokémon. \n\nThe first 151 entries in this Pokédex also serve as [[List of Pokémon by Kanto Pokédex number|Kanto\'s]] [[regional Pokédex]]. In [[Generation II]], this Pokédex order was known as the "Old Pokédex", with a [[List of Pokémon by New Pokédex number|new order]] that 

In [105]:
# FUNTION DEFINITION : get pokemon list
def extract_third_substrings(input_string):
    # Define the regular expression pattern
    # pattern = r"\{\{ndex\|\d+\|([^\|]+)\|[^\|]+\|[^\|]+\}\}"
    # pattern = r"\{\{ndex\|\d+\|([^\|]+)(?:\|[^\|]+)*\}\}"
    pattern = r"\{\{ndex\|\d+\|([^\|]+)\|?.*?\}\}"
    
    # Find all matches of the pattern
    matches = re.findall(pattern, input_string)
    
    # Return the list of third substrings
    return matches

In [112]:
poke_list = extract_third_substrings(poke_list)
poke_list = [name + " (Pokémon)" for name in poke_list]
poke_list

['Bulbasaur (Pokémon)',
 'Ivysaur (Pokémon)',
 'Venusaur (Pokémon)',
 'Charmander (Pokémon)',
 'Charmeleon (Pokémon)',
 'Charizard (Pokémon)',
 'Squirtle (Pokémon)',
 'Wartortle (Pokémon)',
 'Blastoise (Pokémon)',
 'Caterpie (Pokémon)',
 'Metapod (Pokémon)',
 'Butterfree (Pokémon)',
 'Weedle (Pokémon)',
 'Kakuna (Pokémon)',
 'Beedrill (Pokémon)',
 'Pidgey (Pokémon)',
 'Pidgeotto (Pokémon)',
 'Pidgeot (Pokémon)',
 'Rattata (Pokémon)',
 'Raticate (Pokémon)',
 'Spearow (Pokémon)',
 'Fearow (Pokémon)',
 'Ekans (Pokémon)',
 'Arbok (Pokémon)',
 'Pikachu (Pokémon)',
 'Raichu (Pokémon)',
 'Sandshrew (Pokémon)',
 'Sandslash (Pokémon)',
 'Nidoran♀ (Pokémon)',
 'Nidorina (Pokémon)',
 'Nidoqueen (Pokémon)',
 'Nidoran♂ (Pokémon)',
 'Nidorino (Pokémon)',
 'Nidoking (Pokémon)',
 'Clefairy (Pokémon)',
 'Clefable (Pokémon)',
 'Vulpix (Pokémon)',
 'Ninetales (Pokémon)',
 'Jigglypuff (Pokémon)',
 'Wigglytuff (Pokémon)',
 'Zubat (Pokémon)',
 'Golbat (Pokémon)',
 'Oddish (Pokémon)',
 'Gloom (Pokémon)',
 'V

In [113]:
intial_graph = Graph()
for pokemon in poke_list:
    try:
        # Step 1: Fetch the page text
        result1 = bulbabot.page_text(title=pokemon)
        
        # Step 2: Extract the infobox
        result2 = extract_infobox(result1)
        
        # Step 3: Parse the infobox
        result3 = parse_infobox(result2)
        
        # Step 4: Map the parsed infobox to RDF
        final_graph = map_infobox_to_rdf(intial_graph, result3)
        
        # Log or handle the final result (optional)
        print(f"Successfully processed {pokemon}")
    except Exception as e:
        # Handle any errors that occur in the pipeline
        print(f"Error processing {pokemon}: {e}")

Successfully processed Bulbasaur (Pokémon)
Successfully processed Ivysaur (Pokémon)
Successfully processed Venusaur (Pokémon)
Successfully processed Charmander (Pokémon)
Successfully processed Charmeleon (Pokémon)
Successfully processed Charizard (Pokémon)
Successfully processed Squirtle (Pokémon)
Successfully processed Wartortle (Pokémon)
Successfully processed Blastoise (Pokémon)
Successfully processed Caterpie (Pokémon)
Successfully processed Metapod (Pokémon)
Successfully processed Butterfree (Pokémon)
Successfully processed Weedle (Pokémon)
Successfully processed Kakuna (Pokémon)
Successfully processed Beedrill (Pokémon)
Successfully processed Pidgey (Pokémon)
Successfully processed Pidgeotto (Pokémon)
Successfully processed Pidgeot (Pokémon)
Successfully processed Rattata (Pokémon)
Successfully processed Raticate (Pokémon)
Successfully processed Spearow (Pokémon)
Successfully processed Fearow (Pokémon)
Successfully processed Ekans (Pokémon)
Successfully processed Arbok (Pokémon)


In [118]:
# Save or print the created graph
#print(final_graph.serialize(format="turtle"))
with open("poke_graph.ttl", "w", encoding="utf-8") as file:
    file.write(final_graph.serialize(format="turtle"))

In [119]:
Regions_wiht_infobox = [
    "Kanto",
    "Johto",
    "Hoenn",
    "Orre",
    "Sevii Islands",
    "Orange Islands",
    "Pokémon Island",
    "White City",
    "Fiore",
    "Sinnoh",
    "Pokétopia",
    "Pokémon world (Mystery Dungeon)",
    "Mintale Town",
    "Almia",
    "Trading Card Game Islands",
    "Oblivia",
    "Unova",
    "PokéPark (game)",
    "Ransei",
    "Decolore Islands",
    "Kalos",
    "Ferrum",
    "Alola",
    "Carmonte Island",
    "Ryme City",
    "Tumblecube Island",
    "Galar",
    "Pasio",
    "Isle of Armor",
    "Crown Tundra",
    "Lental",
    "Hisui",
    "Aeos Island",
    "Paldea",
    "Kitakami",
    "Blueberry Academy"
]

In [133]:
intial_region_graph = Graph()
for region in Regions_wiht_infobox:
    try:
        # Step 1: Fetch the page text
        result1 = bulbabot.page_text(title=region)
        
        # Step 2: Extract the infobox
        result2 = extract_region_infobox(result1)
        
        # Step 3: Parse the infobox
        result3 = parse_infobox(result2)
        
        # Step 4: Map the parsed infobox to RDF
        final_region_graph = map_region_infobox_to_rdf(intial_region_graph, result3)
        
        # Log or handle the final result (optional)
        print(f"Successfully processed {region}")
    except Exception as e:
        # Handle any errors that occur in the pipeline
        print(f"Error processing {region}: {e}")

Successfully processed Kanto
Successfully processed Johto
Successfully processed Hoenn
Successfully processed Orre
Successfully processed Sevii Islands
Successfully processed Orange Islands
Successfully processed Pokémon Island
Successfully processed White City
Successfully processed Fiore
Successfully processed Sinnoh
Successfully processed Pokétopia
Successfully processed Pokémon world (Mystery Dungeon)
Successfully processed Mintale Town
Successfully processed Almia
Successfully processed Trading Card Game Islands
Successfully processed Oblivia
Successfully processed Unova
Successfully processed PokéPark (game)
Successfully processed Ransei
Successfully processed Decolore Islands
Successfully processed Kalos
Successfully processed Ferrum
Successfully processed Alola
Successfully processed Carmonte Island
Successfully processed Ryme City
Successfully processed Tumblecube Island
Successfully processed Galar
Successfully processed Pasio
Successfully processed Isle of Armor
Successfully

In [ ]:
# Save or print the created graph
print(final_region_graph.serialize(format="turtle"))
# with open("region_graph.ttl", "w", encoding="utf-8") as file:
#     file.write(final_region_graph.serialize(format="turtle"))

@prefix ns1: <http://example.org/region/> .

ns1:Aeos_Island a ns1:Region ;
    ns1:colorTemplate "Orange" ;
    ns1:generation "{{gen|VIII}}" ;
    ns1:image "UNITE Aeos Island.png" ;
    ns1:imageCaption "Aeos Island" ;
    ns1:imageSize "250" ;
    ns1:introductionMedia "[[Pokémon UNITE]]" ;
    ns1:isRegion "no" ;
    ns1:japaneseName "エオス島" ;
    ns1:name "Aeos Island" ;
    ns1:professor "[[Professor Phorus]]" ;
    ns1:romanizedJapaneseName "Aeos Island" .

ns1:Almia a ns1:Region ;
    ns1:colorTemplate "SoA" ;
    ns1:image "Almia.png" ;
    ns1:imageSize "200" ;
    ns1:introductionMedia "[[Pokémon Ranger: Shadows of Almia]]" ;
    ns1:japaneseName "アルミア" ;
    ns1:name "Almia" ;
    ns1:professor "[[Professor Hastings]]" ;
    ns1:romanizedJapaneseName "Almia" ;
    ns1:villainousOrganization "[[Team Dim Sun]]" .

ns1:Alola a ns1:Region ;
    ns1:colorTemplate "alola" ;
    ns1:image "Alola USUM artwork.png" ;
    ns1:imageCaption "Artwork of the Alola region from {{g|Ultra S

In [135]:
with open('character_list.txt', 'r') as file:
    character_list = [line.strip() for line in file]

print(character_list)

['Tracey Sketchit', 'Max', 'Misty', 'Steven Stone', 'Norman', 'Ritchie', 'Giovanni', 'Erika', 'Wes', 'Red (game)', 'Jessie', 'James', 'Lt. Surge', 'Sabrina', 'Blaine', 'Lorelei', 'Lance', 'Nurse Joy', 'Name Rater', 'Todd Snap', 'Leaf (game)', 'Ethan (game)', 'Kris (game)', 'May (game)', 'Anabel', 'Peanut', 'Michael', 'Solana', 'Fantina', 'Lunick', 'Dawn (game)', 'Mark (TCG GB)', 'Mint (TCG GB)', 'Lucy Fleetfoot', 'Mai', 'Marnie', 'PokÃ©mon Center lady', 'Kellyn', 'Kate (Ranger)', 'Koga', 'Ash Ketchum', 'Brock', 'Kanata (CCP)', 'Primo', 'Cynthia', 'Lyra (game)', 'May (anime)', 'Carob', 'Day-Care Couple', 'Dawn (anime)', 'Ben (Ranger)', 'Summer (Ranger)', 'Misty (anime)', 'Brock (anime)', 'Professor Juniper', 'Hilda (game)', 'Hilbert (game)', 'Cheren', 'Bianca', 'N', 'Fennel', 'Iris (anime)', 'Cilan', 'Lenora', 'Cress', 'Chili', 'Ghetsis', 'Iris', 'Clay', 'Drayden', 'Shadow Triad', 'Amanita', 'Giallo', 'Cedric Juniper', 'Hawes', 'Ingo', 'Emmet', 'Charles', 'Anthea and Concordia', 'Gorm',

In [139]:
intial_character_graph = Graph()
for character in character_list:
    try:
        # Step 1: Fetch the page text
        result1 = bulbabot.page_text(title=character)
        
        # Step 2: Extract the infobox
        result2 = extract_character_infobox(result1)
        
        # Step 3: Parse the infobox
        result3 = parse_infobox(result2)
        
        # Step 4: Map the parsed infobox to RDF
        final_character_graph = map_character_infobox_to_rdf(intial_character_graph, result3)
        
        # Log or handle the final result (optional)
        print(f"Successfully processed {character}")
    except Exception as e:
        # Handle any errors that occur in the pipeline
        print(f"Error processing {character}: {e}")

Successfully processed Tracey Sketchit
Successfully processed Max
Successfully processed Misty
Successfully processed Steven Stone
Successfully processed Norman
Successfully processed Ritchie
Successfully processed Giovanni
Successfully processed Erika
Successfully processed Wes
Successfully processed Red (game)
Successfully processed Jessie
Successfully processed James
Successfully processed Lt. Surge
Successfully processed Sabrina
Successfully processed Blaine
Successfully processed Lorelei
Successfully processed Lance
Successfully processed Nurse Joy
Successfully processed Name Rater
Successfully processed Todd Snap
Successfully processed Leaf (game)
Successfully processed Ethan (game)
Successfully processed Kris (game)
Successfully processed May (game)
Successfully processed Anabel
Successfully processed Peanut
Successfully processed Michael
Successfully processed Solana
Successfully processed Fantina
Successfully processed Lunick
Successfully processed Dawn (game)
Successfully pro

In [140]:
# Save or print the created graph
print(final_character_graph.serialize(format="turtle"))
with open("character_graph.ttl", "w", encoding="utf-8") as file:
    file.write(final_character_graph.serialize(format="turtle"))

@prefix ns1: <http://example.org/character/> .

ns1:Aliquis a ns1:Character ;
    ns1:borderColor "D87257" ;
    ns1:color "4B5293" ;
    ns1:coreColor "F1CB90" ;
    ns1:eyeColor "Blue" ;
    ns1:gender "Male" ;
    ns1:hairColor "Dark blue (Yellow highlights)" ;
    ns1:hometown "Unknown" ;
    ns1:image "Aliquis PW.png" ;
    ns1:imageCaption "Aliquis" ;
    ns1:imageSize "220px" ;
    ns1:japaneseName "アリキス" ;
    ns1:name "Aliquis" ;
    ns1:region "[[Paldea]]" ;
    ns1:relatives "[[Aliquis's brother|Unnamed older brother]]<ref>https://www.pokemon.co.jp/ex/sv/ja/sp_anime/</ref>" ;
    ns1:romanizedJapaneseName "Aliquis" ;
    ns1:textColor "FFF" .

ns1:Anabel a ns1:Character ;
    ns1:borderColor "4a474a" ;
    ns1:color "c0b2dd" ;
    ns1:coreColor "e5cff9" ;
    ns1:gender "Female" ;
    ns1:height "5'5\" (1.65 m){{sup/7|SM}}" ;
    ns1:image "Anabel SM concept art.png" ;
    ns1:imageCaption "Concept art from [[Pokémon Sun and Moon]]" ;
    ns1:imageSize "200px" ;
    ns1:japa

In [ ]:
# Define namespaces
POKEMON = Namespace("http://example.org/pokemon/")
RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

# Initialize RDF graph
g = Graph()

# Add Pokémon URI
pokemon_uri = URIRef(POKEMON[properties.get("name", "Unknown").replace(" ", "_")])
g.add((pokemon_uri, RDF.type, POKEMON.Pokémon))

# Map properties to RDF triples
if "species" in properties:
    g.add((pokemon_uri, POKEMON.species, Literal(properties["species"])))

if "type1" in properties:
    g.add((pokemon_uri, POKEMON.type, Literal(properties["type1"])))

if "type2" in properties:
    g.add((pokemon_uri, POKEMON.type, Literal(properties["type2"])))

if "height-m" in properties:
    g.add((pokemon_uri, POKEMON.height, Literal(properties["height-m"])))

if "weight-kg" in properties:
    g.add((pokemon_uri, POKEMON.weight, Literal(properties["weight-kg"])))

# Serialize RDF to Turtle format
print(g.serialize(format="turtle"))

In [73]:
def map_infobox_to_rdf(properties):
    # Define RDF namespaces
    POKEMON = Namespace("http://example.org/pokemon/")
    RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

    # Initialize RDF graph
    g = Graph()

    # Add Pokémon URI
    pokemon_uri = URIRef(POKEMON[properties.get("name", "Unknown").replace(" ", "_")])
    g.add((pokemon_uri, RDF.type, POKEMON.Pokémon))

    # Map general properties
    mappings = {
        "name": POKEMON.name,
        "jname": POKEMON.japaneseName,
        "tmname": POKEMON.trademarkedJapaneseName,
        "jtranslit": POKEMON.japaneseTransliteration,
        "category": POKEMON.category,
        "ndex": POKEMON.nationalDexNumber,
        "forme": POKEMON.numberOfForms,
        "type1": POKEMON.primaryType,
        "type2": POKEMON.secondaryType,
        "ability1": POKEMON.ability1,
        "ability2": POKEMON.ability2,
        "abilityd": POKEMON.hiddenAbility,
        "height-m": POKEMON.heightInMeters,
        "height-ftin": POKEMON.heightInFeetInches,
        "weight-kg": POKEMON.weightInKilograms,
        "weight-lbs": POKEMON.weightInPounds,
        "catchrate": POKEMON.catchRate,
        "gendercode": POKEMON.genderRatioCode,
        "egggroup1": POKEMON.eggGroup1,
        "egggroup2": POKEMON.eggGroup2,
        "color": POKEMON.color,
        "friendship": POKEMON.baseFriendship,
        "generation": POKEMON.generationIntroduced,
        "expyield": POKEMON.baseExperienceYield,
        "evtotal": POKEMON.evTotal,
        "evhp": POKEMON.evYieldHP,
        "evat": POKEMON.evYieldAttack,
        "evde": POKEMON.evYieldDefense,
        "evsa": POKEMON.evYieldSpecialAttack,
        "evsd": POKEMON.evYieldSpecialDefense,
        "evsp": POKEMON.evYieldSpeed
    }

    # Map form-specific properties dynamically
    for i in range(2, 7):  # Forms 2 through 6
        form_suffix = f"form{i}"
        mappings.update({
            f"{form_suffix}": POKEMON[f"form{i}Name"],
            f"{form_suffix}type1": POKEMON[f"form{i}PrimaryType"],
            f"{form_suffix}type2": POKEMON[f"form{i}SecondaryType"],
            f"height-m{i}": POKEMON[f"form{i}HeightInMeters"],
            f"height-ftin{i}": POKEMON[f"form{i}HeightInFeetInches"],
            f"weight-kg{i}": POKEMON[f"form{i}WeightInKilograms"],
            f"weight-lbs{i}": POKEMON[f"form{i}WeightInPounds"],
        })

    # Generate RDF triples for properties that exist in the input
    for key, predicate in mappings.items():
        if key in properties and properties[key]:
            g.add((pokemon_uri, predicate, Literal(properties[key])))

    return g

In [ ]:
grap = map_infobox_to_rdf(properties)
print(grap.serialize(format="turtle"))